In [ ]:
from fastavro import sch

In [7]:
import fastavro

def create_avro_schema(input_dict, schema_name='MySchema'):
    fields = []
    for key, value in input_dict.items():
        field_type = 'string'
        if isinstance(value, int):
            field_type = 'int'
        elif isinstance(value, float):
            field_type = 'float'
        elif isinstance(value, bool):
            field_type = 'boolean'
        elif isinstance(value, list):
            if len(value) > 0:
                field_type = {'type': 'array', 'items': 'string'}
        elif isinstance(value, dict):
            field_type = {'type': 'record', 'name': f'{schema_name}_{key}', 'fields': create_avro_schema(value, schema_name=f'{schema_name}_{key}')['fields']}
        fields.append({'name': key, 'type': [field_type, 'null']})

    schema = {
        'type': 'record',
        'name': schema_name,
        'fields': fields
    }

    return schema

# Example input dictionary
input_dict = {'name': 'John', 'age': 30, 'is_active': True, 'hobbies': ['reading', 'swimming'], 'test': {"1": 2}}

# Generate Avro schema from input dictionary
avro_schema = create_avro_schema(input_dict)

# # Print Avro schema as JSON
# print(fastavro.schemaless_writer_meta(avro_schema))


In [8]:
avro_schema

{'type': 'record',
 'name': 'MySchema',
 'fields': [{'name': 'name', 'type': ['string', 'null']},
  {'name': 'age', 'type': ['int', 'null']},
  {'name': 'is_active', 'type': ['int', 'null']},
  {'name': 'hobbies', 'type': [{'type': 'array', 'items': 'string'}, 'null']},
  {'name': 'test',
   'type': [{'type': 'record',
     'name': 'MySchema_test',
     'fields': [{'name': '1', 'type': ['int', 'null']}]},
    'null']}]}

In [1]:
# Importing libraries

from urllib.request import urlopen, urlretrieve, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm_notebook
from math import ceil
from datetime import datetime
from pathlib import Path

In [2]:
def url_reader(url) -> object:

    """"
    This function reads the html code of the url and returns a BeautifulSoup object.

    Parameters:
    url (str): The url to be read.
    """

    # Need to set user agent to avoid 403 error
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        html = response.read().decode('utf-8')
        soup_obj = BeautifulSoup(html, 'html.parser')
        return soup_obj

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)

In [44]:
soup = url_reader('https://al.olx.com.br/alagoas/imoveis?o=1')

In [45]:
# Gets the number of pages and ads
aux = soup.find('span', class_="sc-1mi5vq6-0 eDXljX sc-bdVaJa juraBY").get_text().split()
ad_per_page = int(aux[2])
n_total_ads = int(aux[-2].replace('.', ''))

number_of_pages=1
# If number_of_pages is not specified, calculate the number of pages
if number_of_pages is not None:
    n_pages = number_of_pages
else:
    n_pages = ceil(n_total_ads/ad_per_page)

In [46]:
ad_list = soup.findAll('a', class_='sc-12rk7z2-1 huFwya sc-giadOv dXANPZ')
ad_links = [ad.get('href') for ad in ad_list]

In [47]:
aux = str(soup.findAll('a', class_='sc-12rk7z2-1 huFwya sc-giadOv dXANPZ')[0])

In [48]:
# regex to get the ad url from the html code
import re
reg1 = r'href="https://al.olx.com.br/alagoas/imoveis/(.*?)"'
regex = re.compile(reg1)
regex.findall(aux)

['casa-em-paripueira-1070176788']

In [49]:
# regex to get the ad url from the html code that ends with a number
reg2 = r'href="https://al.olx.com.br/alagoas/imoveis/(.*?)\d"'
regex2 = re.compile(reg2)
regex2.findall(aux)

['casa-em-paripueira-107017678']

In [41]:
regex2

re.compile(r'href="https://al.olx.com.br/alagoas/imoveis/(.*?)\d"', re.UNICODE)

[]

In [50]:
reg2 = r'href="(https://al.olx.com.br/alagoas/imoveis/.*?)"'
link_list = re.findall(reg2, str(soup))

In [51]:
link_list = [link for link in link_list if link[-1].isdigit()]

In [52]:
link_list

['https://al.olx.com.br/alagoas/imoveis/casa-em-paripueira-1070176788',
 'https://al.olx.com.br/alagoas/imoveis/casa-de-alto-padrao-a-venda-no-condominio-atlantis-4-quartos-4-banheiros-4-vagas-947107441',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-a-venda-em-poco-75-m2-3-quartos-maceio-alagoas-1038171766',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-para-venda-possui-73-metros-quadrados-com-3-quartos-em-poco-maceio-al-1070176498',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-a-venda-na-jatiuca-106m2-3-quartos-3-suites-2-vagas-edificio-gior-978471846',
 'https://al.olx.com.br/alagoas/imoveis/opcao-maravilhosa-na-regiao-mais-cobicada-da-ponta-verde-1070175939',
 'https://al.olx.com.br/alagoas/imoveis/casa-a-vendo-no-laguna-1054753835',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-no-coracao-da-ponta-verde-1020476033',
 'https://al.olx.com.br/alagoas/imoveis/casa-em-condominio-para-venda-em-paripueira-aguas-norte-3-dormitorios-3-banheiros-1064102863',
 'http

In [55]:
# re.findall(r'https://al.olx.com.br/alagoas/imoveis/venda.*"', str(soup))

In [14]:
soup.find_all(re.compile('https://al.olx.com.br/alagoas/imoveis/'))

[]

In [11]:
ad_links

['https://al.olx.com.br/alagoas/imoveis/casa-1069799015',
 'https://al.olx.com.br/alagoas/imoveis/vend-excelente-apartamento-porteira-fechada-dois-quartos-no-farol-1069795756',
 'https://al.olx.com.br/alagoas/imoveis/cobertura-5-4-210m-porteira-fechada-baixou-917217152',
 'https://al.olx.com.br/alagoas/imoveis/casa-a-venda-no-bairro-cidade-universitaria-maceio-al-1069773135',
 'https://al.olx.com.br/alagoas/imoveis/casa-a-venda-no-bairro-cidade-universitaria-maceio-al-1069773117',
 'https://al.olx.com.br/alagoas/imoveis/edificio-divane-luna-apartamentos-a-venda-em-maceio-jatiuca-com-3-quartos-sendo-1-sui-1069772209',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-proximo-ao-shopping-patio-1059504687',
 'https://al.olx.com.br/alagoas/imoveis/apartamentos-de-3-ou-4-suites-na-beira-mar-uma-obra-de-arte-planejado-para-voce-cruz-da-1069771688',
 'https://al.olx.com.br/alagoas/imoveis/vendo-apartamento-medeiros-neto-1-r-70mil-1069767381',
 'https://al.olx.com.br/alagoas/imoveis/casa-5-q

In [62]:
def is_valid(link, base_link):
    return link[-1].isdigit() and link.startswith(f'{base_link}/')

def get_ads_list(soup, base_link):
    regex = fr'href="({base_link}.*?)"'
    link_list = re.findall(regex, str(soup))
    return [link for link in link_list if is_valid(link, base_link)]

In [53]:
url = 'https://al.olx.com.br/alagoas/imoveis?o=1'
soup = url_reader(url)

In [63]:
ad_links = get_ads_list(soup, 'https://al.olx.com.br/alagoas/imoveis')

In [64]:
ad_links

['https://al.olx.com.br/alagoas/imoveis/vendo-casa-no-clima-bom-2-1070693721',
 'https://al.olx.com.br/alagoas/imoveis/casa-antares-cambuci-proximo-a-tudo-1070690127',
 'https://al.olx.com.br/alagoas/imoveis/apt-de-1-quarto-super-luxo-sala-de-estudo-academia-piscina-proximo-a-unit-e-shopping-635370758',
 'https://al.olx.com.br/alagoas/imoveis/aluga-se-uma-casa-1070688551',
 'https://al.olx.com.br/alagoas/imoveis/apart-no-stela-maris-87-metros-1031941120',
 'https://al.olx.com.br/alagoas/imoveis/vendo-otima-vila-no-vilage-9-casas-todas-alugadas-1051290712',
 'https://al.olx.com.br/alagoas/imoveis/jatiuca-vizinho-corredor-vera-arruda-2-qtos-1-ste-e-varanda-area-de-lazer-completa-1009460100',
 'https://al.olx.com.br/alagoas/imoveis/loft-de-praia-em-sao-miguel-dos-milagres-1070242822',
 'https://al.olx.com.br/alagoas/imoveis/casa-sitio-fazenda-prox-frances-1070666918',
 'https://al.olx.com.br/alagoas/imoveis/apartamento-3quartos-serraria-1070666026',
 'https://al.olx.com.br/alagoas/imoveis